In [1]:
import os
import sys
import json
import math
import time
import pickle
import copy
import corner
from datetime import timedelta
from pprint import pprint
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['axes.grid'] = False
matplotlib.rcParams['image.origin'] = 'lower'
import numpy as np
from glob import glob

from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
import astropy.wcs.utils
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.nddata.utils import Cutout2D

from pandeia.engine.perform_calculation import perform_calculation
from pandeia.engine.calc_utils import get_telescope_config, get_instrument_config, build_default_calc, build_default_source
from pandeia.engine.io_utils import read_json, write_json

from lenstronomy.LightModel.Profiles.gaussian import GaussianEllipse
gauss = GaussianEllipse()
import lenstronomy.Plots.plot_util as plot_util
from lenstronomy.SimulationAPI.sim_api import SimAPI
from lenstronomy.ImSim.image_model import ImageModel
from lenstronomy.Data.pixel_grid import PixelGrid
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.Data.psf import PSF
from lenstronomy.SimulationAPI.ObservationConfig.Roman import Roman
from lenstronomy.SimulationAPI.ObservationConfig.HST import HST
from lenstronomy.Util import data_util
from lenstronomy.Plots import lens_plot
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

repo_path = os.getcwd()

data_set_list = glob(repo_path + '/*.png')
print(data_set_list)
execution_times = []

oversample_factor = 1

for i, _ in enumerate(data_set_list):

    execution_start_time = time.time()

    # load lenstronomy model
    model = np.load(f'{i}.npy')

    calc = build_default_calc('roman','wfi','imaging')

    # don't allow scene size to change
    calc['configuration']['dynamic_scene'] = True
    calc['configuration']['max_scene_size'] = 5

    # change filter
    calc['configuration']['instrument']['filter'] = 'f106'

    # adjust brightness (np.interp for now)
    max = np.max(model)
    min = np.min(model)
    model = np.interp(model, (min, max), (0.00001, 0.001))

    j = 0
    side, _ = model.shape

    print('Converting to point sources...')
    for row_number, row in tqdm(enumerate(model), total=side):
        for item_number, item in enumerate(row):
            if j != 0:
                calc['scene'].append(build_default_source(geometry="point"))

            # set brightness
            calc['scene'][j]['spectrum']['normalization']['norm_flux'] = item
            
            # set position
            calc['scene'][j]['position']['x_offset'] = (item_number * (1 / 9) * (1 / oversample_factor)) + (-2.52)  # arcsec
            calc['scene'][j]['position']['y_offset'] = (row_number * (1 / 9) * (1 / oversample_factor)) + (-2.52)  # arcsec

            j += 1
    print('Point source conversion complete')

    print('Performing Pandeia calculation...')
    results = perform_calculation(calc)
    print('Pandeia calculation complete')

    detector = results['2d']['detector']

    # TODO TEMP! flip image
    detector = np.flipud(detector)
    # detector = np.fliplr(detector)

    f, axes = plt.subplots(1, 1, figsize=(6, 6), sharex='none', sharey='none')
    axes.matshow(detector, origin='lower')  # np.log10()
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)
    axes.autoscale(False)
    f.savefig(os.path.join('sim', f'{i}_pandeia_{oversample_factor}.png'))
    plt.close()

    # save this numpy array
    pandeia_array_path = os.path.join('sim', f'{i}_pandeia_{oversample_factor}.npy')
    np.save(pandeia_array_path, detector)

    execution_end_time = time.time()
    execution_time = execution_end_time - execution_start_time
    execution_times.append(execution_time)

plt.scatter(np.arange(0, len(execution_times)), execution_times)
plt.title('Simulation pipeline execution times')
plt.savefig('simulation_pipeline_execution_times.png')
np.save('simulation_pipeline_execution_times', execution_times)


['/nfshome/bwedig/roman-pandeia/arrays/cnn_training/22.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/23.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/0.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/1.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/2.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/3.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/4.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/5.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/6.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/7.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/8.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/9.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/10.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/11.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/12.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_training/13.png', '/nfshome/bwedig/roman-pandeia/arrays/cnn_trainin

100%|██████████| 45/45 [00:10<00:00,  4.46it/s]


Point source conversion complete
Performing Pandeia calculation...


KeyboardInterrupt: 